In [159]:
import pandas as pd
import final_project.utils.paths as path
import numpy
import talib

Analisis de la combinacion de VLM

Leemos el archivo de temporalidad 1 dia

In [160]:
df=pd.read_csv(path.data_processed_dir("SPYVD.csv"))
pd.options.display.float_format='{:,.2f}'.format

Categorizamos las Variables de estudio

In [161]:
df['year']=df.year.astype("category")
df['month']=df.month.astype("category")
df['day']=df.day.astype("category")
df['DayWeek']=df.DayWeek.astype("category")
df['DayStatus']=df.DayStatus.astype("category")
df['BackStatus']=df.BackStatus.astype("category")
df['TBackStatus']=df.TBackStatus.astype("category")
df.convert_dtypes().dtypes

year           category
month          category
day            category
DayWeek        category
OpenOk          Float64
HighOk          Float64
LowOk           Float64
Closeok         Float64
%D              Float64
DayStatus      category
MA200           Float64
MA50            Float64
EMA21           Float64
EMA8            Float64
P/8             boolean
8/21            boolean
21/50           boolean
50/200          boolean
ATRREAL         Float64
MACDHIST        Float64
MACD            Float64
MACDSignal      Float64
Engulfing         Int64
DCC               Int64
Doji              Int64
IHAMER            Int64
HAMER             Int64
2Crows            Int64
3BCrows           Int64
Momentum        Float64
RSI             Float64
BackStatus     category
TBackStatus    category
dtype: object

Agregamos una columna con el número 1 , para contar las variables categoricas
Ademas agragamos columnas con los rangos par los indicadores:
    .Rangos ATR
    .Rangos RSI
    .Rangos Variacion Diaria

In [162]:
df['ones'] = 1
df['Bins_ATR']=pd.cut(df['ATRREAL'],bins=[1,4.5,7,10,13,16])
df['Bins_RSI']=pd.cut(df['RSI'],bins=[15,35,47,60,72,90])
df['Bins_%D']=pd.cut(df['%D'],bins=[-5,-2.8,-1.2,-1,-0.5,0,0.5,1,1.9,4])
##df['BackStatus']=df['DayStatus'].shift(1)
##df['2BackStatus']=df['DayStatus'].shift(2)



Creamos un DataFrame de los dias Martes para el analisis

In [173]:
##dfVLM=df[(df['DayWeek']=='Tuesday')| (df['DayWeek']=='Monday')|(df['DayWeek']=='Friday')]
##| (df['DayWeek']=='Monday')|(df['DayWeek']=='Friday')|(df['DayWeek']=='Wednesday')
dfVLM=df[(df['DayWeek']=='Tuesday')].copy(deep=True)
dfVLMP=df[(df['DayWeek']=='Tuesday')&(df['DayStatus']==1)].copy(deep=True)

In [164]:
dfVLM

,year,month,day,DayWeek,OpenOk,HighOk,LowOk,Closeok,%D,DayStatus,...,2Crows,3BCrows,Momentum,RSI,BackStatus,TBackStatus,ones,Bins_ATR,Bins_RSI,Bins_%D
3,2014,1,7,Tuesday,183.09,183.78,182.95,183.44,0.19,1,...,0,0,NaN,NaN,-1.00,-1.00,1,NaN,NaN,"(0.0, 0.5]"
8,2014,1,14,Tuesday,182.28,183.77,181.95,183.68,0.77,1,...,0,0,NaN,NaN,-1.00,1.00,1,NaN,NaN,"(0.5, 1.0]"
12,2014,1,21,Tuesday,184.69,184.76,183.05,184.19,-0.27,-1,...,0,0,1.80,NaN,-1.00,1.00,1,NaN,NaN,"(-0.5, 0.0]"
17,2014,1,28,Tuesday,178.14,179.29,178.12,179.06,0.52,1,...,0,0,-2.66,38.36,-1.00,-1.00,1,"(1.0, 4.5]","(35.0, 47.0]","(0.5, 1.0]"
22,2014,2,4,Tuesday,174.95,175.83,174.34,175.38,0.25,1,...,0,0,-8.81,35.29,-1.00,1.00,1,"(1.0, 4.5]","(35.0, 47.0]","(0.0, 0.5]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1734,2020,12,1,Tuesday,365.55,367.65,364.93,365.96,0.11,1,...,0,0,3.51,67.08,-1.00,-1.00,1,"(4.5, 7.0]","(60.0, 72.0]","(0.0, 0.5]"
1739,2020,12,8,Tuesday,367.70,370.77,367.67,370.14,0.66,1,...,0,0,12.71,70.10,1.00,1.00,1,"(1.0, 4.5]","(60.0, 72.0]","(0.5, 1.0]"
1744,2020,12,15,Tuesday,367.37,369.57,365.92,369.55,0.59,1,...,0,0,3.59,64.75,-1.00,1.00,1,"(1.0, 4.5]","(60.0, 72.0]","(0.5, 1.0]"
1749,2020,12,22,Tuesday,368.20,368.33,366.03,367.21,-0.27,-1,...,0,0,-2.93,57.02,1.00,-1.00,1,"(1.0, 4.5]","(47.0, 60.0]","(-0.5, 0.0]"


Revisamos las columnas de del DataFrame

In [165]:
dfVLM.columns

Index(['year', 'month', 'day', 'DayWeek', 'OpenOk', 'HighOk', 'LowOk',
       'Closeok', '%D', 'DayStatus', 'MA200', 'MA50', 'EMA21', 'EMA8', 'P/8',
       '8/21', '21/50', '50/200', 'ATRREAL', 'MACDHIST', 'MACD', 'MACDSignal',
       'Engulfing', 'DCC', 'Doji', 'IHAMER', 'HAMER', '2Crows', '3BCrows',
       'Momentum', 'RSI', 'BackStatus', 'TBackStatus', 'ones', 'Bins_ATR',
       'Bins_RSI', 'Bins_%D'],
      dtype='object')

In [166]:
dfVLM.groupby(['DayWeek','DayStatus','BackStatus','TBackStatus'])['ones'].count()/dfVLM.groupby(['DayWeek','DayStatus','BackStatus','TBackStatus'])['ones'].count().sum()*100

DayWeek    DayStatus  BackStatus  TBackStatus
Friday     -1         -1.0        -1.0           0.00
                                  1.0            0.00
                      1.0         -1.0           0.00
                                  1.0            0.00
           1          -1.0        -1.0           0.00
                                  1.0            0.00
                      1.0         -1.0           0.00
                                  1.0            0.00
Monday     -1         -1.0        -1.0           0.00
                                  1.0            0.00
                      1.0         -1.0           0.00
                                  1.0            0.00
           1          -1.0        -1.0           0.00
                                  1.0            0.00
                      1.0         -1.0           0.00
                                  1.0            0.00
Thursday   -1         -1.0        -1.0           0.00
                                  1.

Revisamos DataFrame con los datos analizar en el dia Martes

In [167]:
dfVLM

,year,month,day,DayWeek,OpenOk,HighOk,LowOk,Closeok,%D,DayStatus,...,2Crows,3BCrows,Momentum,RSI,BackStatus,TBackStatus,ones,Bins_ATR,Bins_RSI,Bins_%D
3,2014,1,7,Tuesday,183.09,183.78,182.95,183.44,0.19,1,...,0,0,NaN,NaN,-1.00,-1.00,1,NaN,NaN,"(0.0, 0.5]"
8,2014,1,14,Tuesday,182.28,183.77,181.95,183.68,0.77,1,...,0,0,NaN,NaN,-1.00,1.00,1,NaN,NaN,"(0.5, 1.0]"
12,2014,1,21,Tuesday,184.69,184.76,183.05,184.19,-0.27,-1,...,0,0,1.80,NaN,-1.00,1.00,1,NaN,NaN,"(-0.5, 0.0]"
17,2014,1,28,Tuesday,178.14,179.29,178.12,179.06,0.52,1,...,0,0,-2.66,38.36,-1.00,-1.00,1,"(1.0, 4.5]","(35.0, 47.0]","(0.5, 1.0]"
22,2014,2,4,Tuesday,174.95,175.83,174.34,175.38,0.25,1,...,0,0,-8.81,35.29,-1.00,1.00,1,"(1.0, 4.5]","(35.0, 47.0]","(0.0, 0.5]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1734,2020,12,1,Tuesday,365.55,367.65,364.93,365.96,0.11,1,...,0,0,3.51,67.08,-1.00,-1.00,1,"(4.5, 7.0]","(60.0, 72.0]","(0.0, 0.5]"
1739,2020,12,8,Tuesday,367.70,370.77,367.67,370.14,0.66,1,...,0,0,12.71,70.10,1.00,1.00,1,"(1.0, 4.5]","(60.0, 72.0]","(0.5, 1.0]"
1744,2020,12,15,Tuesday,367.37,369.57,365.92,369.55,0.59,1,...,0,0,3.59,64.75,-1.00,1.00,1,"(1.0, 4.5]","(60.0, 72.0]","(0.5, 1.0]"
1749,2020,12,22,Tuesday,368.20,368.33,366.03,367.21,-0.27,-1,...,0,0,-2.93,57.02,1.00,-1.00,1,"(1.0, 4.5]","(47.0, 60.0]","(-0.5, 0.0]"


Creamos un nuevo DataFrame para analizar cual el la combinacion de los dias Lunes y Viernes anteriores a un Martes

In [174]:
dfVLM_Status=dfVLM.groupby(['DayWeek','DayStatus','BackStatus','TBackStatus'])['ones'].count()/dfVLM.groupby(['DayWeek','DayStatus','BackStatus','TBackStatus'])['ones'].count().sum()*100
dfVLM_StatusP=dfVLMP.groupby(['DayWeek','DayStatus','BackStatus','TBackStatus'])['ones'].count()/dfVLMP.groupby(['DayWeek','DayStatus','BackStatus','TBackStatus'])['ones'].count().sum()*100

Reseteamos los indices

In [175]:
dfVLM_Status=dfVLM_Status.reset_index()
dfVLM_StatusP=dfVLM_StatusP.reset_index()

Generamos una Tabla pivote para analizar la combinacion del dia Viernes y Lunes anteriores al martes en analisis

Cuando los dias Martes son positivo : 56.51%
    Lunes positivo y Vienes Positivo  20.22%
    Lunes positivo y Viernes Negativo 11.63%
    Lunes negativo y Viernes Positivo 12.47%
    Lunes negativo y Viernes Negativo 12.19%


In [172]:
dfVLM_Status.pivot_table(values='ones', index='DayWeek', columns=['DayStatus','BackStatus','TBackStatus'])


DayStatus     -1                      1                  
BackStatus  -1.0        1.0        -1.0         1.0      
TBackStatus -1.0  1.0  -1.0   1.0  -1.0   1.0  -1.0   1.0
DayWeek                                                  
Friday      0.00 0.00  0.00  0.00  0.00  0.00  0.00  0.00
Monday      0.00 0.00  0.00  0.00  0.00  0.00  0.00  0.00
Thursday    0.00 0.00  0.00  0.00  0.00  0.00  0.00  0.00
Tuesday     6.93 9.42 11.36 15.79 12.19 12.47 11.63 20.22
Wednesday   0.00 0.00  0.00  0.00  0.00  0.00  0.00  0.00

In [176]:
dfVLM_StatusP.pivot_table(values='ones', index='DayWeek', columns=['DayStatus','BackStatus','TBackStatus'])

DayStatus     -1                    1                  
BackStatus  -1.0       1.0       -1.0         1.0      
TBackStatus -1.0  1.0 -1.0  1.0  -1.0   1.0  -1.0   1.0
DayWeek                                                
Friday      0.00 0.00 0.00 0.00  0.00  0.00  0.00  0.00
Monday      0.00 0.00 0.00 0.00  0.00  0.00  0.00  0.00
Thursday    0.00 0.00 0.00 0.00  0.00  0.00  0.00  0.00
Tuesday     0.00 0.00 0.00 0.00 21.57 22.06 20.59 35.78
Wednesday   0.00 0.00 0.00 0.00  0.00  0.00  0.00  0.00